#An alternative to Grid Search

Grid Search is a brute force algorithm that helps tunning hyperparameters. Since it is brute force, Grid Search is expensive. To be precise if we want to find the best model out of 1000 different models, using a 5-fold cross validation we need to train 5000 models. 

We present an alternative that uses a hypothesis test (binomial test) and only requires to train 1000 models. Thus, we decrease computational costs by 80%. In general if we use $k$-fold cross validation, we decrease computational costs by $\frac{k-1}{k}$%

#Time complexity
We measure time complexity in **fit** steps. So, any other operation is assumed to run in constant time.

Thus, using Grid Search with $k$-fold validation on $M$ models, the time complexity is $\mathcal{O}(Mk)$.

Our strategy consists in training only $M$ models, thus the time complexity is $\mathcal{O}(M)$. This is why we decrease computational costs by $\frac{k-1}{k}$%

#The implementation

* We use ```binomtest``` from **scipy** to perform the hypothesis test.
* We train 256 instances of ```GradientBoostingClassifier``` from **sklearn**
* The dataset consists of samples of [Hotel reservations](https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset)
* We split the data into train, validation and test sets. The test set will be unseen by our models. We evalute our final model on it.

* We define several functions (repository in construction):

1. ```fit(params,features)``` Fit our (256) models tracking parameters, input features and all metrics we will use

2. ``` bt(models,dataset)``` The binomial test applied to the predictions our models return on a given dataset 

3. ```metric(models)``` A measure of overfitting of models based on binomial test

4. ```drop(models)``` A function to do feature selection over a family of models

5. ```scores(model,features,name)``` Returns a Series containing the scores reached by the model.

* The selected model will be the one that minimizes ```metric(models)```
* We record the time each cell needs to run

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split,GridSearchCV,ParameterGrid


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score,precision_score,recall_score,f1_score,matthews_corrcoef,make_scorer
from sklearn import metrics

#our methods are in a private module
from htoverfit import *

!pip install ipython-autotime
%load_ext autotime


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 2.22 ms (started: 2023-03-07 02:53:22 +00:00)


##Data preprocessing

We will use a Kaggle datset to predict whether a reservation will be canceled. We use one-hote encoding to be able to pass categorical features to ```GradientBoostingClassifier```.  We need to drop some samples since their date is wrong.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/kaggle/Hotel Reservations.csv/Hotel Reservations.csv')
df = df.rename({'arrival_year':'year','arrival_month':'month','arrival_date':'day'},axis=1)
df = df[(df['day']<29)|(df['month']!=2)]
df = df.set_index('Booking_ID')
df['booking_status'] = np.where(df['booking_status']=='Not_Canceled',1,0)

df = pd.get_dummies(df,drop_first=True)
df

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,year,month,day,repeated_guest,...,room_type_reserved_Room_Type 2,room_type_reserved_Room_Type 3,room_type_reserved_Room_Type 4,room_type_reserved_Room_Type 5,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online
Booking_ID,,,,,,,,,,,,,,,,,,,,,
INN00001,2,0,1,2,0,224,2017,10,2,0,...,0,0,0,0,0,0,0,0,1,0
INN00002,2,0,2,3,0,5,2018,11,6,0,...,0,0,0,0,0,0,0,0,0,1
INN00003,1,0,2,1,0,1,2018,2,28,0,...,0,0,0,0,0,0,0,0,0,1
INN00004,2,0,0,2,0,211,2018,5,20,0,...,0,0,0,0,0,0,0,0,0,1
INN00005,2,0,1,1,0,48,2018,4,11,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INN36271,3,0,2,6,0,85,2018,8,3,0,...,0,0,1,0,0,0,0,0,0,1
INN36272,2,0,1,3,0,228,2018,10,17,0,...,0,0,0,0,0,0,0,0,0,1
INN36273,2,0,2,6,0,148,2018,7,1,0,...,0,0,0,0,0,0,0,0,0,1


time: 759 ms (started: 2023-03-07 02:53:23 +00:00)


We divide our dataset: train, validation and test sets.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('booking_status',axis=1),df['booking_status'],random_state=0)
X_train1, X_val, y_train1, y_val = train_test_split(X_train,y_train,random_state=0)

time: 49.1 ms (started: 2023-03-07 02:53:23 +00:00)


##Fit 
We want to choose the best hyperparameters among 256 options. Instead of doing crosvalidation we will fit once our 256 models. In average this step takes 14 minutes. The model selection does not increase significantly the run time. (Our time complexity relies on the fit process).

In [ ]:
params = {'learning_rate': [.01,.04,0.1,0.6],
  'max_depth': [2,3,4,6],
  'min_samples_leaf': [50,100,200,300],
  'n_estimators': [10, 50 , 100,200]}

model = fit(params,df.drop('booking_status',axis=1).columns)

time: 14min 16s (started: 2023-03-07 02:53:23 +00:00)


##Model selection
We take the top 100 models according to a binomial hypothesis test. To be precise we want to test if the distribution of classes predicted by our models matches the expected distribution, the one of the validation targets. 

In [ ]:
p_train = len(y_train1[y_train1==1])/len(y_train1)
p_val = len(y_val[y_val==1])/len(y_val)

bt(model,'validation')

model_vpv = model.copy()
model_vpv.sort(reverse=True,key=lambda x: x['bt_validation'].pvalue)

best_val = model_vpv[0:100]

time: 825 ms (started: 2023-03-07 03:07:40 +00:00)


We take the top 50 models according to a binomial hypothesis test. The same procedure as above, but this time our expected distribution corresponds to train targets.

In [ ]:
bt(best_val,'train')
model_tpv = best_val.copy()
model_tpv.sort(reverse=True,key=lambda x: x['bt_train'].pvalue)

best_train = model_tpv[0:50]

time: 549 ms (started: 2023-03-07 03:07:40 +00:00)


We use the statistics obtained by the binomial tests to determine whether our models are overfit. The key concept is that the binomial test tells us how far is our predicted distribution from the real distribution. We drop those models above the .6 percentile according to our metric.

In [ ]:
metric(best_train)
best_train.sort(key=lambda x: x['overfit'])
winners = best_train[0:20]

time: 1.65 ms (started: 2023-03-07 03:07:41 +00:00)


###Raw Selection

We take the model with the less overfitting. We use its parameters to train a new model on the train and validation sets. We evalute it with the test set. So far our stretegy needed 14 minutes to run.

In [ ]:
best_simple_params = winners[0]['param']
simple_features = winners[0]['features']
simple = GradientBoostingClassifier(**best_simple_params,random_state=0)
simple.fit(X_train,y_train)

simple_scores = scores(simple,features=simple_features,name='Model_binomial_test')
simple_scores

Precision    0.901567
F1           0.919093
ROC_AUC      0.864365
MCC          0.745456
Name: Model_binomial_test, dtype: float64

time: 5.95 s (started: 2023-03-07 03:07:41 +00:00)


###Grid Search
We will compare our model with the model obtained by a grid search using cv. Notice that it take 75 minutes to run the Grid Search. Notice that the 20% of 75 is exactly 15; as we calculated our strategy saves 80% of time.


In [ ]:
params = {'learning_rate': [.01,.04,0.1,0.6],
  'max_depth': [2,3,4,6],
  'min_samples_leaf': [50,100,200,300],
  'n_estimators': [10, 50 , 100,200]}

gb = GradientBoostingClassifier(random_state=0)
gs = GridSearchCV(gb,
                  params,
                  scoring={'precision':make_scorer(metrics.precision_score),
                           'f1':make_scorer(metrics.f1_score),
                           'roc_auc':make_scorer(metrics.roc_auc_score),
                           'mcc':make_scorer(metrics.matthews_corrcoef)},
                  refit=False)
gs.fit(X_train,y_train)

GridSearchCV(estimator=GradientBoostingClassifier(random_state=0),
             param_grid={'learning_rate': [0.01, 0.04, 0.1, 0.6],
                         'max_depth': [2, 3, 4, 6],
                         'min_samples_leaf': [50, 100, 200, 300],
                         'n_estimators': [10, 50, 100, 200]},
             refit=False,
             scoring={'f1': make_scorer(f1_score),
                      'mcc': make_scorer(matthews_corrcoef),
                      'precision': make_scorer(precision_score),
                      'roc_auc': make_scorer(roc_auc_score)})

time: 1h 14min 59s (started: 2023-03-07 03:32:47 +00:00)


Since we are interested in four scores, we must check the best model for each score.

In [ ]:
rank_precision = gs.cv_results_['rank_test_precision']
best_precision = np.where(rank_precision==1)[0]
gs.cv_results_['params'][best_precision[0]]

{'learning_rate': 0.6,
 'max_depth': 6,
 'min_samples_leaf': 50,
 'n_estimators': 200}

time: 5.87 ms (started: 2023-03-07 04:47:46 +00:00)


In [ ]:
rank_f1 = gs.cv_results_['rank_test_f1']
best_f1 = np.where(rank_f1==1)[0]
gs.cv_results_['params'][best_f1[0]]

{'learning_rate': 0.6,
 'max_depth': 6,
 'min_samples_leaf': 50,
 'n_estimators': 200}

time: 5.36 ms (started: 2023-03-07 04:47:46 +00:00)


In [ ]:
rank_roc_auc = gs.cv_results_['rank_test_roc_auc']
best_roc_auc = np.where(rank_roc_auc==1)[0]
gs.cv_results_['params'][best_roc_auc[0]]

{'learning_rate': 0.6,
 'max_depth': 6,
 'min_samples_leaf': 50,
 'n_estimators': 200}

time: 6.79 ms (started: 2023-03-07 04:47:46 +00:00)


In [ ]:
rank_mcc = gs.cv_results_['rank_test_mcc']
best_mcc = np.where(rank_mcc==1)[0]
gs.cv_results_['params'][best_mcc[0]]

{'learning_rate': 0.6,
 'max_depth': 6,
 'min_samples_leaf': 50,
 'n_estimators': 200}

time: 5.15 ms (started: 2023-03-07 04:47:46 +00:00)


In this case, there is a model that maximizes the all four scores. We train it with the original features. We evaluate it on the test set.

In [ ]:
gb = GradientBoostingClassifier(**gs.cv_results_['params'][best_mcc[0]], random_state=0)
gb.fit(X_train,y_train)

gs_scores = scores(gb,features=X_train.columns,name='Model_grid_search')
gs_scores

Precision    0.909631
F1           0.920864
ROC_AUC      0.871790
MCC          0.753918
Name: Model_grid_search, dtype: float64

time: 15.3 s (started: 2023-03-07 04:47:46 +00:00)


### Comparison

Compared with the raw model (no feature selection), the model obtained by Grid Searh performs less than 1% better. In average it performs only .6% better.

In other words, the model obtained using Binomial Test not only saved us 80% of computational costs but it performs just as well as the Grid Search model.

In [ ]:
gs_scores-simple_scores

Precision    0.008064
F1           0.001771
ROC_AUC      0.007425
MCC          0.008462
dtype: float64

time: 13 ms (started: 2023-03-07 04:48:02 +00:00)


### Feature Selection
Since we have saved 80% of computational costs, we can improve our model using feature selection.

Recal ```winners``` stores 20 of the 256 initial models. Now, we improve them with feature selection. In this case we only have 27 features, so we can drop one at a time, the one with the less importance. This increases our complexity time $\mathcal{O}(mf)$ where $m$ is the number of selected models and $f$ is the number of original features. In this case we need to train 20$\times$27=540 extra models.

We add 25 extra minutes, but we still far from the 75 minutes Grid Search needed.

In [ ]:
history = [winners]
begin = winners
while len(begin)>=1:
  next = drop(begin)
  history.append(next)
  begin = next

time: 24min 51s (started: 2023-03-07 03:07:47 +00:00)


Thus, we have a family of models whose input varies in size. We choose the one tha minimizes overfit.

In [ ]:
model_fam = []
for i in range(len(winners)):
  model_i = [x[i] for x in history[:-1]]
  model_i.sort(key=lambda y:y['overfit'])
  model_fam.append(model_i)

best_fam = [x[0] for x in model_fam]
best_fam.sort(key=lambda y:y['overfit'])

time: 3.3 ms (started: 2023-03-07 03:32:39 +00:00)


Now, we train our model on the train and validation sets. Then we calculate four scores.

In [ ]:
overall_best_params = best_fam[0]['param']
overall_best_features = best_fam[0]['features']
overall_best = GradientBoostingClassifier(**overall_best_params,random_state=0)
overall_best.fit(X_train[overall_best_features],y_train)

ob_scores =scores(overall_best,features=overall_best_features,name='Model_binomial_test_feature_selection')
ob_scores

Precision    0.896351
F1           0.916533
ROC_AUC      0.858326
MCC          0.736104
Name: Model_binomial_test_feature_selection, dtype: float64

time: 7.94 s (started: 2023-03-07 03:32:39 +00:00)


Grid search only performs 1% better than the model we found using feature selection.

In [ ]:
gs_scores-ob_scores

Precision    0.013280
F1           0.004332
ROC_AUC      0.013464
MCC          0.017815
dtype: float64

time: 7.94 ms (started: 2023-03-07 04:48:02 +00:00)


### Results

We comfirm that our strategy reduces $\frac{k-1}{k}$% of computational costs. In this case $k=5$. Even more, we noticed that we can improve our model doing feature selection. This improvement does not affect significantly how our model performs and we still save time.

###Conclusions

Hypothesis testing can help decrease the computational expenses involved in identifying a machine learning model.